# Saúde (Health)

In [1]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
import numpy as np
from utils.manual_data import manual_data_path

In [2]:
onibus_diesel_dia31= pd.read_csv(manual_data_path('linestring_2025-12-31_silver.csv'))
df_onibus=onibus_diesel_dia31
df_onibus['geometry'] = df_onibus['geometry'].apply(wkt.loads)
gdf_onibus =gpd.GeoDataFrame(
    df_onibus, 
    geometry= 'geometry',
    crs="EPSG:31983"
)


np.random.seed(20260130)
sample_onibus=gdf_onibus.sample(100).copy()

## Definir valores dos buffers (Defining Buffer values)
#### PT-BR
O valor dos buffers para NOx e MP serão definidos futuramente, após conversarmos com especialistas da área da saúde.
#### ENG
The buffer values for NOx and MP will be defined in the future, after a talk with people specialized in the area

In [3]:
BUFFER_NOX=5
BUFFER_MP=10

In [4]:
gdf_buffer_nox=sample_onibus.copy()
gdf_buffer_nox['geometry'] = sample_onibus.geometry.buffer(BUFFER_NOX)

gdf_buffer_mp=sample_onibus.copy()
gdf_buffer_mp['geometry'] = sample_onibus.geometry.buffer(BUFFER_MP)

## Overlay Setores Censitários

#### PT-BR
Dados dos setores censitários com atributos dos resultados de população e domicílios (Censo de 2022) do estado de São Paulo, extraídos do IBGE.
#### ENG
Census sectors data, with population and households attributes (2022 Census) for São Paulo state, extracted from IBGE

In [5]:
from setores_censitarios import setores_censitarios_final
setores_censitarios_sp = setores_censitarios_final()
setores_censitarios_sp['area_setor'] = setores_censitarios_sp.geometry.area

Retornando arquivo salvo


#### PT-BR
Para calcular a quantidade da população afetada, primeiro é preciso intersecionar a linha do trajeto dos ônibus (com buffer) e os setores censitários, que constam com a informação da quantidade de moradores. Na função `overlay_setores_censitários`, que faz esse cálculo, o geodataframe `gdf_buffer` e `setores_censitarios_sp` são intersecionados, gerando o `gdf_overlay`. Como esse processo pode gerar uma série de interseções muito pequenas, estabelecemos como tamanho mínimo (`debuff_value`) o valor da raíz do `BUFFER` (`(BUFFER**(1/2))`), e isso é calculado usando esse valor mínimo como buffer negativo (`-(BUFFER**(1/2))`). As áreas que forem menores ou iguais ao valor mínimo (`debuff_value`) terão a geometria zerada após a subtração do `debuff_value`, então basta tirar as interseções cuja área<=0 após o debuff.
#### ENG

In [6]:
def overlay_setores_censitarios(
    gdf_buffer, 
    BUFFER,
#    setores_censitarios_sp:gpd.GeoDataFrame()=setores_censitarios_sp
):
    #overlay
    gdf_overlay = gpd.overlay(
        setores_censitarios_sp,
        gdf_buffer,
        how = 'intersection'
    )
    gdf_overlay = gdf_overlay.explode(index_parts = False).reset_index(drop = True)

    #tirar geometrias pequenininhas
    print(f'Lenght before drop: {len(gdf_overlay)}')
    
    debuff_value= -(BUFFER**(1/2))
    gdf_overlay['debuffed'] = gdf_overlay.buffer(debuff_value)
    gdf_overlay['debuffed_area']= gdf_overlay['debuffed'].area
    gdf_overlay = gdf_overlay[gdf_overlay['debuffed_area']>0]
    gdf_overlay = gdf_overlay.drop(columns = {'debuffed', 'debuffed_area'})
    
    print(f' Lenght after drop: {len(gdf_overlay)}')

    return gdf_overlay

In [7]:
gdf_overlay_nox = overlay_setores_censitarios(gdf_buffer_nox, BUFFER_NOX)

gdf_overlay_mp = overlay_setores_censitarios(gdf_buffer_mp, BUFFER_MP)

Lenght before drop: 34946
 Lenght after drop: 31838
Lenght before drop: 30688
 Lenght after drop: 28034


## Cálculo da População Afetada
#### PT-BR
A partir das interseções geradas anteriormente e armazenadas no `gdf_overlay`, calculamos a quantidade de pessoas que estaria em cada interseção, partindo do pressuposto de que a distribuição dos moradores do setor censitário é homogênea, de forma que o cáculo da quantidade de pessoas afetadas seja proporcional ao tamanho da área intersecionada: calculamos a porcentagem que a área da interseção (`area_intersecao`) é da área total do setor (`area_setor`), resultando na coluna `pct_area_intersec`. Determinamos que essa porcentagem da população total do setor é a quantidade de pessoas da interseção (`pop_afetada=pop_setor*pct_area_intersec`)então que a quantidade de pessoas na interseção é equivalente a essa porcentagem da população total. É importante que no final a conta resulte em um número inteiro, já que a nossa unidade de medida são pessoas.
#### ENG

In [8]:
def calculo_pop_afetada(gdf_overlay):

    gdf_overlay['area_intersecao'] = gdf_overlay.geometry.area
    gdf_overlay['pct_area_intersec'] = gdf_overlay['area_intersecao'] / gdf_overlay['area_setor']
    gdf_overlay['pop_afetada'] = round(
        gdf_overlay['pop_setor'] * gdf_overlay['pct_area_intersec']
    ).astype(int)
    
    return gdf_overlay

In [9]:
gdf_overlay_nox =calculo_pop_afetada(gdf_overlay_nox)

gdf_overlay_mp = calculo_pop_afetada(gdf_overlay_mp)

## Agregação por setor e ônibus (Bus and census sector aggregations)
#### PT-BR
Outras formas de agregações foram feitas também (por distrito e linha de ônibus), mas por não serem o foco optei por deixá-las de fora por enquanto.
#### ENG

### Setor Censitário (Census Sector)

In [10]:
# soma MP por setor
mp_sum_setor = (
    gdf_overlay_mp
    .groupby('CD_SETOR', as_index=False)['pop_afetada']
    .sum()
    .rename(columns={'pop_afetada': 'mp_pop_afetada'})
)

# soma NOX por setor
nox_sum_setor = (
    gdf_overlay_nox
    .groupby('CD_SETOR', as_index=False)['pop_afetada']
    .sum()
    .rename(columns={'pop_afetada': 'nox_pop_afetada'})
)

# merge com a tabela principal
setores_censitarios_sp = (
    setores_censitarios_sp
    .merge(
        mp_sum_setor, 
        on='CD_SETOR', 
        how='left'
    ).merge(
        nox_sum_setor, 
        on='CD_SETOR', 
        how='left'
    )
)

### Ônibus (Buses)

In [11]:
mp_sum_onibus = (
    gdf_overlay_mp
    .groupby('codigo_onibus', as_index=False)['pop_afetada']
    .sum()
    .rename(columns={'pop_afetada': 'mp_pop_afetada'})
)


nox_sum_onibus = (
    gdf_overlay_nox
    .groupby('codigo_onibus', as_index=False)['pop_afetada']
    .sum()
    .rename(columns={'pop_afetada': 'nox_pop_afetada'})
)


sample_onibus = (
    sample_onibus
    .merge(
        mp_sum_onibus, 
        on='codigo_onibus', 
        how='left'
    ).merge(
        nox_sum_onibus, 
        on='codigo_onibus', 
        how='left'
    )
)